In [1]:
"""
    This module aims to implement the decomposition of unitary matrices
    using Symmetric Mach-Zehnder Interferometers based on Bell's further
    developed approach of Clement's scheme.
    
    William R. Clements: An Optimal Design for Universal Multiport Interferometers
    B.A. Bell: Further Compactifying Linear Optical Unitaries
    
    The following code was heavily influenced by Clement's interferometer from
    github: https://github.com/clementsw/interferometer
    
    @author: Csaba Szilard Racz //Szyli
"""

import numpy as np


class Beamsplitter:
    """
    This class defines a tunable beam splitter cell
    i.e. a sMZI with internal phase shifters.
    
    ---
    
    The matrix describing the mode transformation is:
    
    >>``e^{i*summ}*sin(delta)``      ``e^{i*summ}cos(delta)``
    
    >>``e^{i*summ}*cos(delta)``     ``-e^{i*summ}sin(delta)``
    
    where
    - ``summ = (theta1 + theta2)/2``
    - ``delta = (theta1 - theta2)/2``
    
    with ``theta1`` and ``theta2`` being the internal phase shifts.
    
    ---

    Args:
        mode1 (int): the index of the first mode (the first mode is mode 1)
        mode2 (int): the index of the second mode
        theta1 (float): angle of internal phase shift on mode1
        theta2 (float): angle of internal phase shift on mode2
    """
    

    def __init__(self, mode1, mode2, theta1, theta2):
        self.mode1 = mode1
        self.mode2 = mode2
        self.theta1 = theta1
        self.theta2 = theta2

    def __repr__(self):
        repr = "\n MZI between modes {} and {}: \n Theta angle on {}: {:.2f} \n the angle on {}: {:.2f}".format(
            self.mode1,
            self.mode2,
            self.mode1,
            self.theta1,
            self.mode2,
            self.theta2,
            )
        return repr


class Interferometer:
    """
    This class defines an interferometer.
    
    ---

    An interferometer contains an ordered list of variable beam splitters,
    represented here by ``BS_list``. For ``BS`` in ``BS_list``, ``BS[0]``
    and ``BS[1]`` correspond to the labels of the two modes being interfered (which start at 1).
    
    This transformation is parametrized by ``BS[2]`` (theta1) and by ``BS[3]`` (theta2)
    which determines the beam splitter reflectivity.
    The interferometer also contains a list of input phases in ``initial_phases``
    that is the only external phaseshifter that is not located at the output.
    Output phases are described by ``output_phases``.
    
    ---
    
    Args:
        BS_list (list): of beamsplitters; elements are of ``Beamsplitter`` class type
        initial_phases (list): of phaseshifts at the beginning of the diagonals
        output_phases (list): of phaseshifts put at the output of the circuit to complete it
    """

    def __init__(self):
        self.BS_list = []
        self.initial_phases = []
        self.output_phases = []

    def add_BS(self, BS):
        """Adds a beam splitter at the output of the current interferometer

        Args:
            BS (Beamsplitter): a Beamsplitter instance
        """
        self.BS_list.append(BS)

    def add_phase(self, mode, phase):    
        """Use this to manually add a phase shift to a selected mode at the output of the interferometer
        
        Args:
            mode (int): the mode index. The first mode is mode 1
            phase (float): the real-valued phase to add
        """
        while mode > np.size(self.output_phases):
            self.output_phases.append(0)
        self.output_phases[mode-1] = phase

    def count_modes(self) -> int:
        """
        Calculate number of modes involved in the transformation. 
        
        ---
        
        This is required for the functions ``calculate_transformation`` and ``draw``.
        
        ---

        Returns:
            the number of modes in the transformation
        """
        highest_index = max([max([BS.mode1, BS.mode2]) for BS in self.BS_list])
        return highest_index

    def calculate_transformation(self) -> np.ndarray:
        """
        Calculate unitary matrix describing the transformation implemented by the interferometer.
        Used to verify the implementation.
    
        Returns:
            complex-valued 2D numpy array representing the interferometer
        """
        N = int(self.count_modes())
        U = np.eye(N, dtype=np.complex_)

        for BS in self.BS_list:
            T = np.eye(N, dtype=np.complex_)
            T[BS.mode1 - 1, BS.mode1 - 1] = np.exp(1j * BS.phi) * np.cos(BS.theta)
            T[BS.mode1 - 1, BS.mode2 - 1] = -np.sin(BS.theta)
            T[BS.mode2 - 1, BS.mode1 - 1] = np.exp(1j * BS.phi) * np.sin(BS.theta)
            T[BS.mode2 - 1, BS.mode2 - 1] = np.cos(BS.theta)
            U = np.matmul(T,U)

        while np.size(self.output_phases) < N:  # Autofill for users who don't want to bother with output phases
            self.output_phases.append(0)

        D = np.diag(np.exp([1j * phase for phase in self.output_phases]))
        U = np.matmul(D,U)
        return U

    def draw(self, show_plot=True):  
        """Use matplotlib to make a drawing of the interferometer

        Args:
            show_plot (bool): whether to show the generated plot
        """

        import matplotlib.pyplot as plt
        plt.figure()
        N = self.count_modes()
        mode_tracker = np.zeros(N)

        for ii in range(N):
            plt.plot((-1, 0), (ii, ii), lw=1, color="blue")

        for BS in self.BS_list:
            x = np.max([mode_tracker[BS.mode1 - 1], mode_tracker[BS.mode2 - 1]])
            plt.plot((x+0.3, x+1), (N - BS.mode1, N - BS.mode2), lw=1, color="blue")
            plt.plot((x, x+0.3), (N - BS.mode1, N - BS.mode1), lw=1, color="blue")
            plt.plot((x, x+0.3), (N - BS.mode2, N - BS.mode2), lw=1, color="blue")
            plt.plot((x+0.3, x+1), (N - BS.mode2, N - BS.mode1), lw=1, color="blue")
            plt.plot((x+0.4, x+0.9), (N - (BS.mode2 + BS.mode1)/2, N - (BS.mode2 + BS.mode1)/2), lw=1, color="blue")
            reflectivity = "{:2f}".format(np.cos(BS.theta)**2)
            plt.text(x+0.9, N + 0.05 - (BS.mode2 + BS.mode1)/2, reflectivity[0:3], color="green", fontsize=7)

            plt.plot((x+0.15, x+0.15), (N+0.3-(BS.mode2 + BS.mode1)/2., N+0.7-(BS.mode2 + BS.mode1)/2.), lw=1, color="blue")
            circle = plt.Circle((x+0.15, N+0.5-(BS.mode2 + BS.mode1)/2.), 0.1, fill=False)
            plt.gca().add_patch(circle)
            phase = "{:2f}".format(BS.phi)
            if BS.phi > 0:
                plt.text(x+0.2, N+0.7-(BS.mode2 + BS.mode1)/2., phase[0:3], color="red", fontsize=7)
            else:
                plt.text(x+0.2, N+0.7-(BS.mode2 + BS.mode1)/2., phase[0:4], color="red", fontsize=7)
            if x > mode_tracker[BS.mode1-1]:
                plt.plot((mode_tracker[BS.mode1-1], x), (N-BS.mode1, N-BS.mode1), lw=1, color="blue")
            if x > mode_tracker[BS.mode2-1]:
                plt.plot((mode_tracker[BS.mode2-1], x), (N-BS.mode2, N-BS.mode2), lw=1, color="blue")
            mode_tracker[BS.mode1-1] = x+1
            mode_tracker[BS.mode2-1] = x+1

        max_x = np.max(mode_tracker)
        for ii in range(N):
            plt.plot((mode_tracker[ii], max_x+1), (N-ii-1, N-ii-1), lw=1, color="blue")
            while np.size(self.output_phases) < N:  # Autofill for users who don't want to bother with output phases
                self.output_phases.append(0)
            if self.output_phases[ii] != 0:
                plt.plot((max_x+0.5, max_x+0.5), (N-ii-1.2, N-ii-0.8), lw=1, color="blue")
                circle = plt.Circle((max_x+0.5, N-ii-1), 0.1, fill=False)
                plt.gca().add_patch(circle)
                phase = str(self.output_phases[ii])
                if BS.phi > 0:
                    plt.text(max_x+0.6, N-ii-0.8, phase[0:3], color="red", fontsize=7)
                else:
                    plt.text(max_x+0.6, N-ii-0.8, phase[0:4], color="red", fontsize=7)


        plt.text(max_x/2, -0.7, "green: BS reflectivity", color="green", fontsize=10)
        plt.text(max_x/2, -1.4, "red: phase shift", color="red", fontsize=10)
        plt.text(-1, N-0.3, "Light in", fontsize=10)
        plt.text(max_x+0.5, N-0.3, "Light out", fontsize=10)
        plt.gca().axes.set_ylim([-1.8, N+0.2])
        plt.axis("off")
        if show_plot:
            plt.show()


def square_decomposition(U):
    """
    Returns a rectangular mesh of beam splitters implementing matrix U.
    
    ---

    This code implements the decomposition algorithm in:
    

    Returns:
        an Interferometer instance
    """
    I = Interferometer()
    m = U.shape[0]    # dimension of matrix = number of modes 'm'
    V = np.conjugate(U)
    even = []
    circuit = np.zeros((m,m),dtype=complex) #each entry will define the phase that needs to be put at the corresponding layer a and mode b
    ext_PS_in = np.zeros((m),dtype=complex) #the external PS P applied to the input
    ext_PS_out = np.zeros((m),dtype=complex) #the external PS P applied to the output
    
    for j in range(1, m):
        # odd diags: 1,3,5...
        if j%2 != 0: # ii%2
            x = m-1
            y = j-1
            s = y+1 #place of the external phase shift P 
            # find external phaseshift that matches given elements' phases
            P = external_ps(m, s, V[x,y], V[x,y+1])

            ext_PS_in[s] = np.angle(V[x,y])-np.angle(V[x,y+1])
            V = np.matmul(V,P)
           # print(np.angle(V[x,y])-np.angle(V[x,y+1]))
            
            for k in range(1, j+1):
                modes = [y, y+1]    # initial mode-pairs
                
                delta = custom_arctan(V[x,y+1], V[x,y])
                
                if k == j:
                    # redundant choice (?)
                    summ = 0
                else:
                    # derivation shows
                    summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta))
                
                M = np.eye(m, dtype=np.complex_)
                M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
                M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
                M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
                M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)
                V = np.matmul(V,M)
                
                theta1, theta2 = internal_phases(delta,summ)
                
                #save the angles in the matrix circuit to easily read where to put which phase
                a, b = MZI_layer_coord(m,modes,j,k)
                circuit[a,b] = theta1
                circuit[a,b+1] = theta2

                
                I.BS_list.append(Beamsplitter(modes[0], modes[1], theta1, theta2))
                # print("j,k: {:.2f},{:.2f}\nnulled: {:.2f}".format(j,k,V[x,y]))
                
                # update coordinates
                x -= 1
                y -= 1
                
        # even numbered diagonals (j = 2,4,6...)
        else:
            x = m-j
            y = 0
            s = x-1 #place of the external phase shift P
            
            P = external_ps(m, s, V[x,y], V[x-1,y])

            ext_PS_out[s] = np.angle(V[x,y])-np.angle(V[x-1,y])
            V = np.matmul(P,V)
          
            for k in range(1, j+1): # jj
                modes = [x-1, x]     # initial mode-pairs
                
                delta = custom_arctan(-V[x-1,y], V[x,y])
                if k == j:
                    summ = 0
                else:
                    # derivation shows
                    summ = np.angle(V[x+1,y+1]) - np.angle(V[x-1,y+1]*np.cos(delta) - V[x,y+1]*np.sin(delta))
                
                M = np.eye(m, dtype=np.complex_)
                M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
                M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
                M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
                M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)
                V = np.matmul(M,V)
                
                #calculate the actual phases theta1 and theta2
                theta1, theta2 = internal_phases(delta,summ)
                
                #save the angles in the matrix circuit to easily read where to put which phase
                a, b = MZI_layer_coord(m,modes,j,k)
                circuit[a,b] = theta1
                circuit[a,b+1] = theta2
                
                even.append(Beamsplitter(modes[0], modes[1], theta1, theta2))
                
                # print("j,k: {:.2f},{:.2f}\nnulled: {:.2f}".format(j,k,V[x,y]))
                
                # update coordinates
                x += 1
                y += 1

    #add step 3 of the algorithm that implements the external phases in the middle of the cicuit
    #in addiditon we now want to move the external phases Q to the residual positions
    
    #this phase shift on the first mode ensures that we implement the actual unitary U, without global phase, 
    #however, for now we don't use it
    #V = np.dot(V,external_ps(m, 0, 0, V[0,0]))
    
    if m%2 != 0: #if the number of modes is odd
        for j in range(2,m+1):
            #xi = np.angle(V[0][0])-np.angle(V[j-1][j-1])
            a = m - j
            #print(j,a)
            xi = np.angle(V[0][0])-np.angle(V[j-1][j-1]) 
   
            if j%2 != 0:
                #if j is odd        
                for b in range(j-1,m):
                    circuit[a,b] = circuit[a,b] + xi
                for b in range(j,m):
                    circuit[a-1,b] = circuit[a-1,b] - xi
                
            else: #if j is even           
                for b in range(j):
                    circuit[a-1,b] = circuit[a-1,b] + xi
                for b in range(j-1):
                    circuit[a,b] = circuit[a,b] - xi
            
                
            V = np.dot(V,external_ps(m, j-1, V[0,0], V[j-1,j-1]))
            
    else: #for even m
        for j in range(2,m+1):
            a = m - j
            xi = np.angle(V[0][0])-np.angle(V[j-1][j-1]) 
   
            if j%2 != 0:
                #if j is odd        
                for b in range(j):
                    circuit[a,b] = circuit[a,b] + xi
                for b in range(j-1):
                    circuit[a+1,b] = circuit[a+1,b] - xi
                    
            else: #if j is even           
                for b in range(j-1,m):
                    circuit[a+1,b] = circuit[a+1,b] + xi
                for b in range(j,m):
                    circuit[a,b] = circuit[a,b] - xi
            
                
            V = np.dot(V,external_ps(m, j-1, V[0,0], V[j-1,j-1]))
                    
                
        
    #add the even MZIs to the BS list:
    for BS in np.flip(even, 0):
        I.BS_list.append(BS)
    
    # for BS in np.flip(left_T, 0):
    #     modes = [int(BS.mode1), int(BS.mode2)]
    #     invT = np.eye(N, dtype=np.complex_)
    #     invT[modes[0]-1, modes[0]-1] = np.exp(-1j * BS.phi) * np.cos(BS.theta)
    #     invT[modes[0]-1, modes[1]-1] = np.exp(-1j * BS.phi) * np.sin(BS.theta)
    #     invT[modes[1]-1, modes[0]-1] = -np.sin(BS.theta)
    #     invT[modes[1]-1, modes[1]-1] = np.cos(BS.theta)
    #     U = np.matmul(invT, U)
    #     theta = custom_arctan(U[modes[1]-1, modes[0]-1], U[modes[1]-1, modes[1]-1])
    #     phi   =  custom_angle(U[modes[1]-1, modes[0]-1], U[modes[1]-1, modes[1]-1])
    #     invT[modes[0]-1, modes[0]-1] = np.exp(-1j * phi) * np.cos(theta)
    #     invT[modes[0]-1, modes[1]-1] = np.exp(-1j * phi) * np.sin(theta)
    #     invT[modes[1]-1, modes[0]-1] = -np.sin(theta)
    #     invT[modes[1]-1, modes[1]-1] = np.cos(theta)
    #     U = np.matmul(U, invT) 
    #     I.BS_list.append(Beamsplitter(modes[0], modes[1], theta, phi))
    # # output (external) phases
    # phases = np.diag(U)
    # I.output_phases = [np.angle(i) for i in phases]
    return ext_PS_in, circuit.T, ext_PS_out
    #return I
    
    #return V


def random_unitary(N: int) -> np.ndarray:
    """
    Returns a random NxN unitary matrix

    This code is inspired by Matlab code written by Toby Cubitt:
    http://www.dr-qubit.org/matlab/randU.m

    Args:
        N (int): dimension of the NxN unitary matrix to generate

    Returns:
        complex-valued 2D numpy array representing the interferometer
    """
    X = np.zeros([N, N], dtype=np.complex_)
    for ii in range(N):
        for jj in range(N):
            X[ii, jj] = (np.random.normal() + 1j * np.random.normal()) / np.sqrt(2)

    q, r = np.linalg.qr(X)
    r = np.diag(np.divide(np.diag(r), abs(np.diag(r))))
    U = np.matmul(q, r)

    return U

def external_ps(N: int, j: int, V1: np.complex_, V2: np.complex_) -> np.ndarray:
    """
    Builds the external phase-shifter for the given diagonal.
    Purpose of this operator is to match the given elements'
    phases.
    
    Parameters
    ------
    N : dimension of unitary matrix / number of modes
    j : selected "diagonal"
    V1 : element of auxillary matrix
    V2 : subsequent element of **V1**
    
    ---
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    
    Returns
    ------
    Diagonal matrix with phase-shift ``exp(i*phi)``
    at position ``[j,j]``.
    """
    phi = np.angle(V1) - np.angle(V2)
    P = np.eye(N, dtype=np.complex_)
    P[j,j] = np.exp(1j * phi)
    
    return P

def phase_match(V1: np.complex_, V2: np.complex_):
    """
    Phase matcher function. Used to find ``summ``.
    
    ---
    
    Parameters
    ------
    V1 : element of auxillary matrix at ``[x+1,y+1]`` or ``[x-1,y-1]``
    V2 : element of auxillary matrix at ``[x,y+1]`` or ``[x-1,y]``
    
    Returns
    ------
    The angle for ``summ``
    """
    phi1 = np.angle(V1)
    phi2 = np.angle(V2)
    
    print('angle1: ', phi1, '\nangle2: ', phi2)
    
def custom_arctan(V1, V2):
    """
    Computes the ``arctan`` of ``-V1/V2``.
    
    ---
    If ``V2=0`` returns ``pi/2``.
    
    """
    if V2 != 0:
        return np.arctan(-V1/V2)
    else:
        return np.pi/2

def custom_angle(x1, x2):
    if x2 != 0:
        return np.angle(x1/x2)
    else:
        return 0
    
def internal_phases(delta,summ):
    """
    Computes the internal phases theta1 and theta2 
    
    ------
    
    Parameters
    ------
    delta: the angle computed to null the elements
    summ: the phase computed to equalize the phases
    
    Returns
    ------
    The interal phases 'theta1' and 'theta2', that reproduce 'delta' and 'summ' according to
    summ = (theta1+theta2)/2
    detla = (theta1-theta2)/2
    In addition we add a phase -pi/2 to each angle to ensure the proper description of the BS
    """
    theta1 = delta + summ - np.pi/2
    theta2 = summ - delta - np.pi/2
    
    return theta1, theta2

def MZI_layer_coord(m,modes,j,k):
    """
    Gives new coordinates that define in which vertical layer of the circuit 
    the corresponding MZI is placed
    
    ------
    
    Parameters
    ------
    m: the number of modes
    modes: the modes the MZI is acting on
    j: the coordinate giving the diagonal of the circuit
    k: the counter within the diagonal j
    
    Returns
    ------
    New coordinates a and b (starting at 0) that will help to place the MZI in the circiut 
    and manage the shifting of the external PS Q from the middle of the circuit to the residual positions
    a: the coordinate of the MZI layer
    b: the affected mode
    """
    
    if j%2 == 0 :
        a = m - k
    else:
        a = k - 1
    b = modes[0]
        
    return a, b

def matprod(mat_list):
    """
    Computes the product of the matrices in the list
    
    ------
    
    Parameters
    ------
    mat_list: a list of matrices that shall be mulitplied in the given order
    
    Returns
    ------
    A matrix that is the product of all matrices in mat_list
    """
    result = np.eye(mat_list[0].shape[0])
    for i in mat_list:
        result = np.dot(result, i)
    return result

def BS(m,m1,m2):
    """
    Gives a matrix that descibes a 50/50 beam splitter acting on modes m1 and m1 embedded in a mxm matrix
    
    ------
    
    Parameters
    ------
    m: the number of modes
    m1: mode 1 the BS acts on
    m1: mode 2 the BS acts on
    Returns
    ------
    An mxm matrix with embedded 50/50 BS on modes m1 and m2
    """
    B = np.eye(m,dtype=complex)
    B[m1,m1] = 1/(2**0.5)
    B[m1,m1+1] = 1j/(2**0.5)
    B[m2,m2-1] = 1j/(2**0.5)
    B[m2,m2] = 1/(2**0.5)
    
    return B

def PS(phases_list):
    """
    Gives a matrix that describes phase shifts on the corresponding modes
    
    ------
    
    Parameters
    ------
    phases_list: a list of phases that should be applied to the corresponding modes 
    
    Returns
    ------
    A diagonal matrix with the phase shifts on its diagonal
    """
    exps = [np.exp(1j*p) for p in phases_list]
    P = np.diag(exps)
    return P

def test_func(m,In,Out,C,U):
    """
    Tests if the calculated phases give back the target unitary
    
    ------
    
    Parameters
    ------
    m: the number of modes
    In: list of phase shifts at the input of the circuit
    Out: list of phase shifts at the output of the circuit
    C: matrix that contains all the phases that should be implemented in the circuit
    U: target unitary matrix
    
    Returns
    ------
    Computes the matrix that is implemented using the phases given in In, Out and C and compares it
    to the target unitary U. If they are the same the decomposition works, 
    if they differ, the difference is printed for furhter error analysis.
    Note: this function can also be used to get the unitary, that is implemented by a given circuit,
    if it has the respective geometry
    """
    #there are two different layers of BS, one starts at mode 0, the other at mode 1
    Beam_sp1 = []
    i = 0
    while i < m - 1:
        Beam_sp1.append(BS(m,i,i+1))
        i = i + 2
    layerA = matprod(Beam_sp1)   
    
    Beam_sp2 = []    
    i = 1
    while i < m - 1:
        Beam_sp2.append(BS(m,i,i+1))
        i = i + 2
    layerB = matprod(Beam_sp2)
    
    #we alternate between the two different BS layers and sandwich the phase shifts
    #since for an even number of modes we have a slightly different geometry, one has to treat the cases differently 
    U_list = []
    
    if m%2 != 0:
        for j in range(m):
            if j%2 == 0:
                U_list.append(matprod([layerA,PS(C[:,m-1-j]),layerA]))
            else:
                U_list.append(matprod([layerB,PS(C[:,m-1-j]),layerB]))
    else:
        for j in range(m):
            if j%2 == 0:
                U_list.append(matprod([layerB,PS(C[:,m-1-j]),layerB]))
            else:
                U_list.append(matprod([layerA,PS(C[:,m-1-j]),layerA])) 
    
    #multiply all BS layers and PS in the inner circuit (all described in the output matrix C)        
    U_mat = matprod(U_list)
    
    #add the external PS to the input and output 
    U_imp = matprod([PS(Out),U_mat,PS(In)])
    
    #compare with the desired unitary U, they should be equal up to a global phase, so we renormalize the test matrix
    test = np.dot(U,U_imp.T.conj())
    test = test/test[0][0]
    
    if np.allclose(test,np.eye(m,dtype=complex)):
        print("Function Works!")
    else:
        print(test-np.eye(m,dtype=complex))


In [2]:
def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ', suppress_small=True)}")
    print('\n')

Latest updates:
* Nulling works properly after adapting the places of the external PS P
* added step 3 of the algorithm which gives the external PS Q
* added function that computes the actual angles theta1 and theta2
* the interferometer instance gives a list of MZIs with the phases theta and their respective modes, so far one has to read each diagonal from bottom left to top right, also the PS Q are not included in the list -> does it make sense to add them, since in step 2 we want to remove them to the residual places, also we have to think in which order or format we want this output list in order to combine it nicely with the new software
* adapted the angles theta1 and theta2 such that the BS now can be described by [[1,i],[i,1]]/2**0.5

## Manual Tests (old) 

In [253]:
t1 = np.random.rand()
t2 = np.random.rand()
B = np.array([[1,1j],[1j,1]])/(2**0.5)
P = np.array([[np.exp((t1)*1j),0],[0,np.exp((t2)*1j)]])


In [254]:
t1 = t1+np.pi/2
t2 = t2+np.pi/2
summ = (t1+t2)/2
delta = (t1-t2)/2

In [255]:
M = np.array([[np.sin(delta) * np.exp(1j*summ),np.cos(delta) * np.exp(1j*summ)],[np.cos(delta) * np.exp(1j*summ),-np.sin(delta) * np.exp(1j*summ)]])


In [256]:
(np.dot(np.dot(B,P),B)-M).round(10)

array([[ 0.+0.j,  0.-0.j],
       [ 0.-0.j, -0.-0.j]])

In [241]:
#test if the computed phases give back the original unitary
#start for 3x3 case

B01 = np.array([[1/(2**0.5),1j/(2**0.5),0],[1j/(2**0.5),1/(2**0.5),0],[0,0,1]])
B12 = np.array([[1,0,0],[0,1/(2**0.5),1j/(2**0.5)],[0,1j/(2**0.5),1/(2**0.5)]])

def PS(t1,t2,t3):
    return np.array([[np.exp(t1*1j),0,0],[0,np.exp(t2*1j),0],[0,0,np.exp(t3*1j)]])

In [242]:
B12.round(5)

array([[1.     +0.j     , 0.     +0.j     , 0.     +0.j     ],
       [0.     +0.j     , 0.70711+0.j     , 0.     +0.70711j],
       [0.     +0.j     , 0.     +0.70711j, 0.70711+0.j     ]])

In [243]:
U = random_unitary(3)
In,C,Out = square_decomposition(U)

In [151]:
MZI1 = np.dot(np.dot(B01,PS(C[0,0],C[1,0],C[2,0])),B01)
MZI2 = np.dot(np.dot(B12,PS(C[0,1],C[1,1],C[2,1])),B12)
MZI3 = np.dot(np.dot(B01,PS(C[0,2],C[1,2],C[2,2])),B01)

MZI1 = np.dot(PS(In[0],In[1],In[2]),MZI1)
MZI2 = np.dot(PS(0,0,0),MZI2)
MZI3 = np.dot(MZI3,PS(Out[0],Out[1],Out[2]))

V = np.dot(np.dot(MZI1,MZI2),MZI3)
np.dot(V,U.T.conj()).round(10)
#(U-V).round(2)
#(V*(0.92789174+0.37284972j)-U).round(5)

array([[-0.69833279+0.71577322j, -0.        -0.j        ,
        -0.        -0.j        ],
       [ 0.        -0.j        , -0.69833279+0.71577322j,
         0.        +0.j        ],
       [-0.        -0.j        ,  0.        -0.j        ,
        -0.69833279+0.71577322j]])

## Write a proper test function 

First, we define the components needed to describe the circuit:

In [9]:
def matprod(mat_list):
    """
    Computes the product of the matrices in the list
    
    ------
    
    Parameters
    ------
    mat_list: a list of matrices that shall be mulitplied in the given order
    
    Returns
    ------
    A matrix that is the product of all matrices in mat_list
    """
    result = np.eye(mat_list[0].shape[0])
    for i in mat_list:
        result = np.dot(result, i)
    return result

In [10]:
# 50/50 BS on modes m1 and m1:
def BS(m,m1,m2):
    """
    Gives a matrix that descibes a 50/50 beam splitter acting on modes m1 and m1 embedded in a mxm matrix
    
    ------
    
    Parameters
    ------
    m: the number of modes
    m1: mode 1 the BS acts on
    m1: mode 2 the BS acts on
    Returns
    ------
    An mxm matrix with embedded 50/50 BS on modes m1 and m2
    """
    B = np.eye(m,dtype=complex)
    B[m1,m1] = 1/(2**0.5)
    B[m1,m1+1] = 1j/(2**0.5)
    B[m2,m2-1] = 1j/(2**0.5)
    B[m2,m2] = 1/(2**0.5)
    
    return B

In [11]:
def PS(phases_list):
    """
    Gives a matrix that describes phase shifts on the corresponding modes
    
    ------
    
    Parameters
    ------
    phases_list: a list of phases that should be applied to the corresponding modes 
    
    Returns
    ------
    A diagonal matrix with the phase shifts on its diagonal
    """
    exps = [np.exp(1j*p) for p in phases_list]
    P = np.diag(exps)
    return P

In [66]:
def test_func(m,In,Out,C,U):
    """
    Tests if the calculated phases give back the target unitary
    
    ------
    
    Parameters
    ------
    m: the number of modes
    In: list of phase shifts at the input of the circuit
    Out: list of phase shifts at the output of the circuit
    C: matrix that contains all the phases that should be implemented in the circuit
    U: target unitary matrix
    
    Returns
    ------
    Computes the matrix that is implemented using the phases given in In, Out and C and compares it
    to the target unitary U. If they are the same the decomposition works, 
    if they differ, the difference is printed for furhter error analysis.
    Note: this function can also be used to get the unitary, that is implemented by a given circuit,
    if it has the respective geometry
    """
    #there are two different layers of BS, one starts at mode 0, the other at mode 1
    Beam_sp1 = []
    i = 0
    while i < m - 1:
        Beam_sp1.append(BS(m,i,i+1))
        i = i + 2
    layerA = matprod(Beam_sp1)   
    
    Beam_sp2 = []    
    i = 1
    while i < m - 1:
        Beam_sp2.append(BS(m,i,i+1))
        i = i + 2
    layerB = matprod(Beam_sp2)
    
    #we alternate between the two different BS layers and sandwich the phase shifts
    #since for an even number of modes we have a slightly different geometry, one has to treat the cases differently 
    U_list = []
    
    if m%2 != 0:
        for j in range(m):
            if j%2 == 0:
                U_list.append(matprod([layerA,PS(C[:,m-1-j]),layerA]))
            else:
                U_list.append(matprod([layerB,PS(C[:,m-1-j]),layerB]))
    else:
        for j in range(m):
            if j%2 == 0:
                U_list.append(matprod([layerB,PS(C[:,m-1-j]),layerB]))
            else:
                U_list.append(matprod([layerA,PS(C[:,m-1-j]),layerA])) 
    
    #multiply all BS layers and PS in the inner circuit (all described in the output matrix C)        
    U_mat = matprod(U_list)
    
    #add the external PS to the input and output 
    U_imp = matprod([PS(Out),U_mat,PS(In)])
    
    #compare with the desired unitary U, they should be equal up to a global phase, so we renormalize the test matrix
    test = np.dot(U,U_imp.T.conj())
    test = test/test[0][0]
    
    if np.allclose(test,np.eye(m,dtype=complex)):
        print("Function Works!")
    else:
        print(test-np.eye(m,dtype=complex))


In [8]:
m=5
U = random_unitary(m)

In,C,Out = square_decomposition(U)
test_func(m,In,Out,C,U)

Function Works!


In [9]:
square_decomposition(U)[1].round(2).real

array([[-1.76,  0.11, -2.35,  0.06, -0.71],
       [-1.6 , -3.74, -0.92,  0.69, -3.54],
       [ 0.81, -1.56,  5.59, -0.54,  4.1 ],
       [ 3.  , -5.42,  4.67, -1.03,  2.95],
       [ 1.57, -6.72,  4.5 , -2.11,  0.  ]])

In [4]:
m=5
U = random_unitary(m)

square_decomposition(U).round(10)

array([[ 0.96227594-0.27207539j,  0.        +0.j        ,
         0.        -0.j        ,  0.        +0.j        ,
         0.        -0.j        ],
       [-0.        +0.j        ,  0.96227594-0.27207539j,
        -0.        -0.j        ,  0.        -0.j        ,
        -0.        -0.j        ],
       [-0.        +0.j        ,  0.        -0.j        ,
         0.96227594-0.27207539j, -0.        +0.j        ,
         0.        +0.j        ],
       [-0.        +0.j        ,  0.        -0.j        ,
         0.        -0.j        ,  0.96227594-0.27207539j,
        -0.        +0.j        ],
       [-0.        +0.j        , -0.        -0.j        ,
         0.        -0.j        ,  0.        +0.j        ,
         0.96227594-0.27207539j]])

## Old Stuff 

In [200]:
m=5
U = random_unitary(m)

In,C,Out = square_decomposition(U)
BS1 = np.dot(BS(m,0,1),BS(m,2,3))
BS2 = np.dot(BS(m,1,2),BS(m,3,4))

1
-1.565828195294887
(-2.4562761101700694-5.797198226395574e-17j) (-0.6853165434197237+5.797198226395574e-17j)
2
5.31908688883727
(-1.5691998722841314+1.2887592659268099e-17j) (-1.7398155451454098-1.2887592659268099e-17j)
(-0.7435601982610084-2.5181826835124928e-17j) (-2.3980324553287846+2.5181826835124928e-17j)
3
-2.1751118582418387
(-4.372678503668492-2.2720702424236232e-17j) (-1.7192745012048396+2.2720702424236232e-17j)
(-0.30145920436593565-5.214431093632962e-17j) (1.5731434150073036+5.214431093632962e-17j)
(-1.967878123349278+4.744700720635728e-17j) (-1.1737145302405152-4.744700720635728e-17j)
4
1.1896561675472999
(1.589310416087646+0j) (-0.259631845040841+0j)
(0.48224904556298265-6.976744031715919e-17j) (-0.7245422080131995+6.976744031715919e-17j)
(0.2897687016734034+9.435946863199047e-17j) (-0.418140576126909-9.435946863199047e-17j)
(-1.0924942651599086-7.996260547684508e-17j) (-2.0490983884298846+7.996260547684508e-17j)
2
-0.4788083633974396
3
-3.734734441386306
2
2
4
-2.414511

In [215]:
MZI1 = np.dot(np.dot(BS1,PS(C[:,0])),BS1)
MZI2 = np.dot(np.dot(BS2,PS(C[:,1])),BS2)
MZI3 = np.dot(np.dot(BS1,PS(C[:,2])),BS1)
MZI4 = np.dot(np.dot(BS2,PS(C[:,3])),BS2)
MZI5 = np.dot(np.dot(BS1,PS(C[:,4])),BS1)
matrix = [PS(In),MZI1,MZI2,MZI3,MZI4,MZI5,PS(Out)]

def matprod(mat_list):
    result = np.eye(mat_list[0].shape[0])
    for i in mat_list:
        result = np.dot(result, i)
    return result

def smatprod(mat_list):
    I = np.eye(mat_list[0].shape[0])
    def dmatprod(mat_list):
        l = len(mat_list)
        if l == 2:
            return np.dot(mat_list[0], mat_list[1])
        if l == 1:
            return mat_list[0]
        if l % 2 == 0:
            mat_list.append(I)
            l +=1
        return np.dot(dmatprod([mat_list[:int(l/2)]]),dmatprod([mat_list[:int(l/2)]]))
    return dmatprod(mat_list)

In [217]:
smatprod(matrix) == matprod(matrix)

<ipython-input-217-f70b06123419>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  smatprod(matrix) == matprod(matrix)


False